In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation

# Procesamiento digital de imágenes

Una imagen es una colección de pixeles ordenados

En estándar RGB cada pixel corresponde a 3 valores enteros de 8 bit (256 niveles). Combinándolos formamos colores (aproximadamente 16.7M)

Otra codificación usual para los pixeles consiste en usar un número entre cero y uno para cada canal (color)

El estándar RGBA añade un canal que representa la opacidad

Las imágenes en escala de grises y sin opacidad se pueden representar usando un canal

In [ ]:
img = plt.imread('img/cameraman.png')

def to_grayscale(img):
    return np.dot(img[:, :, :3], 
                  np.array([0.2989, 0.587, 0.114]))

img_bw = to_grayscale(img)

display(img_bw.shape)
display(img_bw.dtype)

plt.figure(figsize=(4, 4), tight_layout=True)
plt.imshow(img_bw, cmap=plt.cm.Greys_r);

Para nuestros sistemas digitales una imagen es una arreglo multidimensional y podemos operarlo como tal

¿A qué corresponde este segmento del arreglo?

In [ ]:
subimg = np.copy(img_bw[50:100, 120:180])
display(subimg)

In [ ]:
plt.figure(figsize=(3, 3), tight_layout=True)
plt.imshow(subimg, cmap=plt.cm.Greys_r);

¿Y este segmento?

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(6, 3), tight_layout=True)
ax[1].plot(subimg[30, :])
ax[0].imshow(subimg[30:31, :], cmap=plt.cm.Greys_r);

## Convolución  y correlación cruzada discreta

Una herramienta clásica de procesamiento digital de señales es la **convolución**

La operación de convolución entre dos señales unidimensionales discretas es

$$
(f * g) [n] = \sum_{m=-\infty}^\infty  f[m] g[n-m]
$$

y la operación de correlación cruzada es

$$
(f \star g) [n] = \sum_{m=-\infty}^\infty  f[m] g[m+n]
$$

> Para ambas operaciones el resultado es una nueva señal que también depende de  $n$



Por ejemplo el elemento $0$ de $f\star g$ se calcula como

    f[0] g[0] + f[1] g[1] + f[2] g[2] + ...

Luego el elemento $1$ sería

    f[0] g[1] + f[1] g[2] + f[2] g[3] + ...
    
¿Cómo se ve esta operación graficamente?

In [ ]:
import scipy.signal
fig, ax = plt.subplots(2, figsize=(7, 4))
ax2 = ax[0].twinx()

def filt(k):
    kernel = np.zeros(shape=(len(data),))
    kernel[k:k+5] = 1
    #kernel[k] = 1.; kernel[k+1] = -1.
    return kernel

data = subimg[0, :]
true_filt = filt(0)[np.absolute(filt(0)) >0]
display(true_filt)
conv_s = scipy.signal.correlate(data, true_filt, mode='valid')


def update(k): 
    ax[0].cla(); ax[1].cla(); ax2.cla();
    ax[0].plot(data)
    ax2.plot(filt(k), c='r')
    ax[1].plot(conv_s); 
    ax[1].scatter(k, conv_s[k], s=100, c='k')
    
anim = animation.FuncAnimation(fig, update, frames=len(conv_s), interval=200, blit=True)

## Filtrado de imágenes con convoluciones

Se puede extender el concepto de convolución a dos dimensiones
$$
(I_1 * I_2) [n_1, n_2] = \sum_{m_1=-\infty}^\infty \sum_{m_2=-\infty}^\infty I_1[m_1, m_2] I_2[n_1-m_2, n_2 - m_2]
$$

donde $n_1$ es el índice de las filas y $n_2$ es el índice de las columnas

#### La convolución entre dos imágenes es una nueva imagen

La imagen $I_1$ es la entrada

La imagen $I_2$ se denomina filtro o kernel de la convolución

La imagen resultante es la imagen filtrada

#### Filtro pasa-bajo

Suaviza, elimina los detalles

In [ ]:
D = 3
filt = np.ones(shape=(D, D))

display(filt)
img_res = scipy.signal.correlate2d(subimg, filt/np.sum(filt), mode='valid')

fig, ax = plt.subplots(1, 3, figsize=(8, 3), tight_layout=True)
ax[0].imshow(subimg, cmap=plt.cm.Greys_r)
ax[1].imshow(filt, cmap=plt.cm.Greys_r)
ax[2].imshow(img_res, cmap=plt.cm.Greys_r);

#### Filtro pasa-alto

Resalta los cambios bruscos, elimina las partes "planas"

In [ ]:
filt = np.array([[1., -1.]]*2)
display(filt)
img_res = scipy.signal.correlate2d(subimg, filt, mode='valid')

fig, ax = plt.subplots(1, 3, figsize=(8, 3), tight_layout=True)
ax[0].imshow(subimg, cmap=plt.cm.Greys_r)
ax[1].imshow(filt, cmap=plt.cm.Greys_r)
ax[2].imshow(img_res, cmap=plt.cm.Greys_r);

#### ¿Detector de patillas?

Detecta patillas de fotografos mirando al horizonte

In [ ]:
filt = np.ones(shape=(11, 11))
filt[:9, 2:9] = 0
display(filt)

In [ ]:
img_res = scipy.signal.correlate2d(subimg, filt-np.mean(filt), mode='valid')

fig, ax = plt.subplots(1, 3, figsize=(8, 3), tight_layout=True)
ax[0].imshow(subimg, cmap=plt.cm.Greys_r)
maxloc = np.unravel_index(np.argmax(img_res), shape=img_res.shape)
ax[0].scatter(maxloc[1]+filt.shape[0]//2, maxloc[0]+filt.shape[1]//2, c='r', s=20)
ax[1].imshow(filt, cmap=plt.cm.Greys_r)
ax[2].imshow(img_res, cmap=plt.cm.Reds);

En realidad el filtro se activa con cualquier cosa con forma de "U"

El punto es:

> Podríamos aprender filtros para detectar objetos específicos

En ese caso:

> Necesitamos aprender los valores de los "píxeles" del kernel

# Visión computacional

La [visión computacional](http://szeliski.org/Book/) es un campo de investigación que busca que los computadores sean capaces de "comprender" el contenido presente en imágenes digitales y video

#### Objetivo: 

> Automatizar tareas realizas por el sistema visual humano

- Clasificación y Reconocimiento: ¿A qué categoría corresponde el patrón en la imagen?
- Detección, Localización y Segmentación: ¿Dónde está el patrón en la imagen? 
- [Estimación de pose](https://modelzoo.co/blog/deep-learning-models-and-code-for-pose-estimation)
- [Reconstrucción](https://www.youtube.com/watch?v=gg0F5JjKmhA), [super-resolución](https://www.extremetech.com/extreme/132950-csi-style-super-resolution-image-enlargment-yeeaaaah) y [síntesis](https://tcwang0509.github.io/pix2pixHD/)
- ...

<a href="https://towardsdatascience.com/detection-and-segmentation-through-convnets-47aa42de27ea"><img src="https://miro.medium.com/max/800/1*SNvD04dEFIDwNAqSXLQC_g.jpeg" width="600"></a>


#### Aplicaciones

- [Medicina](https://www.rsipvision.com/medical-segmentation/)
- [Navegación autónoma](https://www.youtube.com/watch?v=H7Ym3DMSGms)
- [Sistemas de control de tráfico](https://www.youtube.com/watch?v=jxhAWuImxS8)
- [Realidad aumentada](https://www.youtube.com/watch?v=r9hVypi_6TQ)
- [Agricultura y forestal](https://medium.com/@awangenh/mapping-weeds-and-crops-in-precision-agriculture-with-convolutional-neural-networks-138dab87ba00)
- [...](https://www.cs.ubc.ca/~lowe/vision.html)

#### Herramientas

- Procesamiento digital de imágenes
- Optimización, Estadística 
- Machine learning y en particular  **Redes Neuronales Convolucionales** 


#### Desafios

- Algoritmos invariantes a los cambios de Iluminación
- Algoritmos invariantes a los cambios de escala y perspectiva (deformación)
- Algoritmos robustos contra la oclusión